## Import package and Setting

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
import pathlib

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "14"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)

## Load image data

In [3]:
data_path = "flower_photos/"

In [4]:
data_root = pathlib.Path(data_path)

In [5]:
label_names = sorted(item.name for item in data_root.glob("*/") if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("label names:", label_names)
print("label to index:", label_to_index)

label names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
label to index: {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [6]:
all_image_paths = list(data_root.glob("*/*"))
all_image_paths = [str(path) for path in all_image_paths]

In [7]:
random.shuffle(all_image_paths)

In [8]:
train_image_paths = all_image_paths[500:]
valid_image_paths = all_image_paths[:500]

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [10]:
def preprocessing_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image -= tf.reduce_mean(image)
    image /= (tf.math.reduce_std(image)+1e-5)
    return image

In [11]:
def load_and_preprocessing_image(path):
    image = tf.io.read_file(path)
    return preprocessing_image(image)

In [12]:
def dataset_generate(image_paths, batch_size, label_to_index, AUTOTUNE):
    image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in image_paths]
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(load_and_preprocessing_image, num_parallel_calls=AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(image_labels, tf.float32))
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
    image_counts = len(image_paths)
    print("Step:", int(image_counts/batch_size))
    image_label_ds = image_label_ds.shuffle(buffer_size=image_counts).repeat().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
    return image_label_ds

In [13]:
train_datasets = dataset_generate(train_image_paths, 16, label_to_index, AUTOTUNE)
valid_datasets = dataset_generate(valid_image_paths, 16, label_to_index, AUTOTUNE)

Step: 198
Step: 31


## Build Model

In [14]:
model = tf.keras.Sequential()
conv_base = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))
model.add(conv_base)
model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Dense(5, activation="softmax"))

In [15]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["acc"])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


## Training

In [17]:
model.fit(train_datasets, steps_per_epoch=300, 
          validation_data=valid_datasets, validation_steps=50,
          epochs=30, verbose=2, shuffle=True)

Train for 300 steps, validate for 50 steps
Epoch 1/30
300/300 - 32s - loss: 1.0972 - acc: 0.6310 - val_loss: 1.8844 - val_acc: 0.2525
Epoch 2/30
300/300 - 20s - loss: 0.6237 - acc: 0.7821 - val_loss: 1.1433 - val_acc: 0.5713
Epoch 3/30
300/300 - 19s - loss: 0.3930 - acc: 0.8640 - val_loss: 2.3754 - val_acc: 0.5763
Epoch 4/30
300/300 - 19s - loss: 0.3944 - acc: 0.8660 - val_loss: 0.8235 - val_acc: 0.7375
Epoch 5/30
300/300 - 19s - loss: 0.2226 - acc: 0.9229 - val_loss: 0.9750 - val_acc: 0.7387
Epoch 6/30
300/300 - 19s - loss: 0.2109 - acc: 0.9273 - val_loss: 1.0229 - val_acc: 0.7262
Epoch 7/30
300/300 - 19s - loss: 0.1780 - acc: 0.9369 - val_loss: 0.8266 - val_acc: 0.7613
Epoch 8/30
300/300 - 19s - loss: 0.1350 - acc: 0.9544 - val_loss: 0.8854 - val_acc: 0.7950
Epoch 9/30
300/300 - 19s - loss: 0.1109 - acc: 0.9654 - val_loss: 0.4620 - val_acc: 0.8363
Epoch 10/30
300/300 - 19s - loss: 0.0862 - acc: 0.9712 - val_loss: 1.3141 - val_acc: 0.7212
Epoch 11/30
300/300 - 19s - loss: 0.1147 - acc

## Save model

In [18]:
model.save("flower.h5", include_optimizer=False)